In [1]:
# 🎭 한국 전래동화 캐릭터 음성 데이터 준비 (구글 드라이브 버전)
# Google Colab에서 실행

import os
import shutil
from pathlib import Path
import json

# ========================================
# STEP 0: 구글 드라이브 마운트
# ========================================

print("📂 구글 드라이브 연결 중...")

from google.colab import drive
drive.mount('/content/drive')

print("✅ 구글 드라이브 연결 완료!")


📂 구글 드라이브 연결 중...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 구글 드라이브 연결 완료!


In [2]:

# ========================================
# STEP 1: 소스 폴더 경로 설정
# ========================================

print("\n" + "="*60)
print("STEP 1: 데이터 위치 확인")
print("="*60)

# 구글 드라이브의 audio_sample 폴더 경로
# 실제 경로에 맞게 수정하세요
source_dir = "/content/drive/MyDrive/P_dev/audio_sample"

# 경로 확인
if not os.path.exists(source_dir):
    print(f"\n⚠️ 폴더를 찾을 수 없습니다: {source_dir}")
    print("\n구글 드라이브 구조를 확인하세요:")
    print("  1. 왼쪽 사이드바 '파일' 아이콘 클릭")
    print("  2. drive > MyDrive 하위 경로 확인")
    print("  3. 위 source_dir 경로를 실제 경로로 수정")
    raise FileNotFoundError(f"폴더 없음: {source_dir}")

print(f"\n✅ 소스 폴더 발견: {source_dir}")

# 폴더 목록 확인
voice_folders = sorted([f for f in os.listdir(source_dir)
                       if os.path.isdir(os.path.join(source_dir, f))])

print(f"\n📋 발견된 성우 폴더 ({len(voice_folders)}개):")
for folder in voice_folders:
    folder_path = os.path.join(source_dir, folder)
    wav_count = len(list(Path(folder_path).glob("*.wav")))
    print(f"  ✓ {folder}: {wav_count}개 WAV 파일")

# ========================================
# STEP 2: 캐릭터 매핑 정의 (12개)
# ========================================

print("\n" + "="*60)
print("STEP 2: 캐릭터 매핑 설정")
print("="*60)

# 전체 12개 캐릭터 매핑
character_mapping = {
    # 남성 캐릭터
    "boy": "M0101_명량한 소년",
    "scholar": "M0202_선비",
    "nolbu": "M0303_놀부",
    "king": "M0404_왕",
    "commander": "M0505_지휘관",
    "bachelor": "M0606_총각",

    # 여성 캐릭터
    "teacher": "W0107_선생님",
    "librarian": "W0208_사서",
    "queen": "W0309_여왕",
    "girl": "W0410_소녀",
    "grandma": "W0511_할머니",
    "princess": "W0612_공주"
}

print("\n🎭 캐릭터 목록:")
print("\n[남성 캐릭터]")
male_chars = ["boy", "scholar", "nolbu", "king", "commander", "bachelor"]
for char in male_chars:
    print(f"  • {char:12s} ← {character_mapping[char]}")

print("\n[여성 캐릭터]")
female_chars = ["teacher", "librarian", "queen", "girl", "grandma", "princess"]
for char in female_chars:
    print(f"  • {char:12s} ← {character_mapping[char]}")

# ========================================
# STEP 3: GPT-SoVITS용 데이터셋 구성
# ========================================

print("\n" + "="*60)
print("STEP 3: 데이터셋 재구성")
print("="*60)

output_base = "/content/GPT-SoVITS/dataset"
os.makedirs(output_base, exist_ok=True)

print("\n📁 파일 복사 중...")

for character, folder_name in character_mapping.items():
    src_folder = os.path.join(source_dir, folder_name)
    dst_folder = os.path.join(output_base, character)

    # 폴더 없으면 건너뛰기
    if not os.path.exists(src_folder):
        print(f"  ⚠️ {character}: 소스 폴더 없음 ({folder_name})")
        continue

    os.makedirs(dst_folder, exist_ok=True)

    # WAV 파일 복사 및 이름 변경
    wav_files = list(Path(src_folder).glob("*.wav"))

    for i, wav_file in enumerate(wav_files):
        # 통일된 파일명: character_0001.wav
        new_name = f"{character}_{i:04d}.wav"
        dst_path = os.path.join(dst_folder, new_name)

        shutil.copy2(str(wav_file), dst_path)

    print(f"  ✅ {character:12s}: {len(wav_files):3d}개 파일")

print("\n✅ 데이터셋 구성 완료!")

# ========================================
# STEP 4: 데이터 통계 확인
# ========================================

print("\n" + "="*60)
print("STEP 4: 데이터 품질 검사")
print("="*60)

# librosa 설치
print("\n📦 필요한 라이브러리 설치 중...")
!pip install librosa soundfile -q

import librosa
import numpy as np

print("\n📊 캐릭터별 음성 데이터 통계:\n")

total_minutes = 0
stats_list = []

for character in character_mapping.keys():
    char_dir = os.path.join(output_base, character)

    if not os.path.exists(char_dir):
        continue

    wav_files = list(Path(char_dir).glob("*.wav"))

    if not wav_files:
        print(f"  ⚠️ {character:12s}: 파일 없음")
        continue

    # 샘플 10개로 평균 계산 (시간 절약)
    sample_files = wav_files[:min(10, len(wav_files))]
    durations = []
    sample_rates = []

    for wav_file in sample_files:
        try:
            y, sr = librosa.load(str(wav_file), sr=None)
            duration = librosa.get_duration(y=y, sr=sr)
            durations.append(duration)
            sample_rates.append(sr)
        except Exception as e:
            print(f"    ⚠️ 파일 로드 실패: {wav_file.name}")

    if durations:
        avg_duration = np.mean(durations)
        estimated_total = avg_duration * len(wav_files)
        minutes = estimated_total / 60
        total_minutes += minutes

        # 통계 저장
        stats_list.append({
            'character': character,
            'files': len(wav_files),
            'minutes': minutes,
            'avg_sr': int(np.mean(sample_rates))
        })

        # 상태 표시
        if minutes >= 10:
            status = "✅ 우수"
        elif minutes >= 5:
            status = "✅ 양호"
        else:
            status = "⚠️ 부족"

        print(f"  {status} {character:12s}: {minutes:5.1f}분 ({len(wav_files):3d}개, {int(np.mean(sample_rates))}Hz)")

print(f"\n📈 전체 통계:")
print(f"  • 총 캐릭터: {len(stats_list)}개")
print(f"  • 총 음성 길이: {total_minutes:.1f}분 ({total_minutes/60:.1f}시간)")
print(f"  • 캐릭터당 평균: {total_minutes/len(stats_list):.1f}분")

# 권장사항
insufficient = [s for s in stats_list if s['minutes'] < 5]
if insufficient:
    print(f"\n⚠️ 5분 미만 캐릭터 ({len(insufficient)}개):")
    for s in insufficient:
        print(f"    • {s['character']}: {s['minutes']:.1f}분")
    print("  → 추가 음성 데이터 수집 권장")

# ========================================
# STEP 5: 참조 오디오 정보 생성
# ========================================

print("\n" + "="*60)
print("STEP 5: 참조 오디오 설정")
print("="*60)

# 각 캐릭터의 대표 참조 텍스트
reference_texts = {
    "boy": "나는 씩씩한 소년이야!",
    "scholar": "학문의 길은 멀고도 험하도다.",
    "nolbu": "이놈, 감히 형님한테 대들어!",
    "king": "짐이 명하노니, 모두 물러가거라.",
    "commander": "전군, 집결하라!",
    "bachelor": "저는 총각입니다.",
    "teacher": "자, 이제 공부를 시작해볼까요?",
    "librarian": "조용히 하세요. 여기는 도서관입니다.",
    "queen": "나는 이 나라의 여왕이니라.",
    "girl": "엄마, 나 배고파!",
    "grandma": "옛날 옛날에 말이야.",
    "princess": "저는 아름다운 공주랍니다."
}

reference_info = {}

print("\n🎯 각 캐릭터의 참조 오디오 선정:\n")

for character in character_mapping.keys():
    char_dir = os.path.join(output_base, character)

    if not os.path.exists(char_dir):
        continue

    wav_files = sorted(list(Path(char_dir).glob("*.wav")))

    if wav_files:
        # 첫 번째 파일을 참조로 사용
        ref_audio = str(wav_files[0])

        reference_info[character] = {
            "ref_audio_path": ref_audio,
            "ref_text": reference_texts.get(character, "안녕하세요"),
            "voice_type": character_mapping[character],
            "total_files": len(wav_files)
        }

        print(f"  {character:12s}: {wav_files[0].name}")

# JSON 저장
ref_json_path = "/content/GPT-SoVITS/character_references.json"
with open(ref_json_path, 'w', encoding='utf-8') as f:
    json.dump(reference_info, f, ensure_ascii=False, indent=2)

print(f"\n✅ 참조 정보 저장: {ref_json_path}")

# ========================================
# STEP 6: 슬라이싱 생략 - 복사
# ========================================

print("\n" + "="*60)
print("STEP 6: 슬라이싱 단계 생략")
print("="*60)

sliced_base = "/content/GPT-SoVITS/sliced"

print("\n🔄 dataset → sliced 복사 중...")

if os.path.exists(output_base):
    shutil.copytree(output_base, sliced_base, dirs_exist_ok=True)
    print("✅ 복사 완료!")
    print(f"  경로: {sliced_base}")

# ========================================
# STEP 7: 완료 및 다음 단계 안내
# ========================================

print("\n" + "="*60)
print("✅ 모든 데이터 준비 완료!")
print("="*60)

print("\n📂 생성된 폴더 구조:")
print("  /content/GPT-SoVITS/")
print("    ├── dataset/        (원본)")
print("    │   ├── boy/")
print("    │   ├── scholar/")
print("    │   ├── nolbu/")
print("    │   └── ... (12개)")
print("    └── sliced/         (학습용)")
print("        ├── boy/")
print("        └── ... (12개)")

print("\n📄 생성된 파일:")
print(f"  • {ref_json_path}")

print("\n" + "="*60)
print("🎯 다음 단계: ASR 실행")
print("="*60)

print("\n아래 코드를 복사해서 새 셀에서 실행하세요:\n")
print("-" * 60)
print("""
# ASR (음성 → 텍스트 자동 생성)
characters = [
    "boy", "scholar", "nolbu", "king", "commander", "bachelor",
    "teacher", "librarian", "queen", "girl", "grandma", "princess"
]

for char in characters:
    print(f'\\n🗣️ {char} ASR 진행 중...')

    !cd /content/GPT-SoVITS && python tools/asr/funasr_asr.py \\
        -i /content/GPT-SoVITS/sliced/{char} \\
        -o /content/GPT-SoVITS/asr/{char} \\
        -s large \\
        -l ko

    print(f'✅ {char} ASR 완료')

print('\\n✅ 모든 캐릭터 ASR 완료!')
""")
print("-" * 60)

# 추가 정보 출력
print("\n" + "="*60)
print("📌 중요 정보")
print("="*60)
print(f"\n• 총 캐릭터 수: {len(character_mapping)}개")
print(f"• 총 음성 길이: {total_minutes:.1f}분")
print(f"• ASR 예상 소요 시간: {len(character_mapping) * 5}~{len(character_mapping) * 10}분")

print("\n💡 선택적 학습 권장:")
print("  12개 모두 학습하면 15-20시간 소요")
print("  주요 캐릭터 7개만 선택 권장:")
print("  → princess, boy, grandma, king, girl, scholar, nolbu")

print("\n✨ 준비 완료! ASR을 실행하세요.")


STEP 1: 데이터 위치 확인

✅ 소스 폴더 발견: /content/drive/MyDrive/P_dev/audio_sample

📋 발견된 성우 폴더 (12개):
  ✓ M0101_명량한 소년: 7개 WAV 파일
  ✓ M0202_선비: 8개 WAV 파일
  ✓ M0303_놀부: 5개 WAV 파일
  ✓ M0404_왕: 4개 WAV 파일
  ✓ M0505_지휘관: 10개 WAV 파일
  ✓ M0606_총각: 7개 WAV 파일
  ✓ W0107_선생님: 7개 WAV 파일
  ✓ W0208_사서: 14개 WAV 파일
  ✓ W0309_여왕: 4개 WAV 파일
  ✓ W0410_소녀: 13개 WAV 파일
  ✓ W0511_할머니: 5개 WAV 파일
  ✓ W0612_공주: 8개 WAV 파일

STEP 2: 캐릭터 매핑 설정

🎭 캐릭터 목록:

[남성 캐릭터]
  • boy          ← M0101_명량한 소년
  • scholar      ← M0202_선비
  • nolbu        ← M0303_놀부
  • king         ← M0404_왕
  • commander    ← M0505_지휘관
  • bachelor     ← M0606_총각

[여성 캐릭터]
  • teacher      ← W0107_선생님
  • librarian    ← W0208_사서
  • queen        ← W0309_여왕
  • girl         ← W0410_소녀
  • grandma      ← W0511_할머니
  • princess     ← W0612_공주

STEP 3: 데이터셋 재구성

📁 파일 복사 중...
  ✅ boy         :   7개 파일
  ✅ scholar     :   8개 파일
  ✅ nolbu       :   5개 파일
  ✅ king        :   4개 파일
  ✅ commander   :  10개 파일
  ✅ bachel

In [3]:
# 🎙️ GPT-SoVITS 완전 설치 및 학습 파이프라인
# Google Colab에서 실행

# ========================================
# STEP 0: 런타임 확인
# ========================================

print("="*60)
print("STEP 0: 환경 확인")
print("="*60)

import torch
print(f"\n✓ Python 버전: {torch.__version__}")
print(f"✓ CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✓ CUDA 버전: {torch.version.cuda}")

# ========================================
# STEP 1: Python 3.10 설치 (중요!)
# ========================================

print("\n" + "="*60)
print("STEP 1: Python 3.10 설치")
print("="*60)

# Colab은 Python 3.10이므로 건너뛰기
!python --version

# ========================================
# STEP 2: FFmpeg 설치
# ========================================

print("\n" + "="*60)
print("STEP 2: FFmpeg 설치")
print("="*60)

!apt-get update -qq
!apt-get install -y ffmpeg

!ffmpeg -version | head -n 1

# ========================================
# STEP 3: GPT-SoVITS 클론
# ========================================

print("\n" + "="*60)
print("STEP 3: GPT-SoVITS 다운로드")
print("="*60)

import os
os.chdir('/content')

!git clone https://github.com/RVC-Boss/GPT-SoVITS.git
%cd GPT-SoVITS

# ========================================
# STEP 4: PyTorch 설치 (먼저!)
# ========================================

print("\n" + "="*60)
print("STEP 4: PyTorch 설치")
print("="*60)

!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121

# CUDA 확인
import torch
print(f"\n✓ PyTorch 설치 완료: {torch.__version__}")
print(f"✓ CUDA: {torch.cuda.is_available()}")

# ========================================
# STEP 5: 의존성 설치
# ========================================

print("\n" + "="*60)
print("STEP 5: 의존성 설치")
print("="*60)

# pyopenjtalk 제거 (한국어에 불필요)
!sed -i '/pyopenjtalk/d' requirements.txt

# 의존성 설치
!pip install -r requirements.txt

# 추가 패키지
!pip install cn2an pypinyin funasr==1.0.27

print("\n✅ 의존성 설치 완료!")

# ========================================
# STEP 6: 사전 학습 모델 다운로드
# ========================================

print("\n" + "="*60)
print("STEP 6: 사전 학습 모델 다운로드")
print("="*60)

# 디렉토리 생성
!mkdir -p GPT_SoVITS/pretrained_models
!mkdir -p tools/uvr5/uvr5_weights

# Git LFS 설치
!apt-get install git-lfs -y
!git lfs install

# 모델 다운로드 (약 3GB, 시간 소요)
print("\n⏳ 모델 다운로드 중... (3-5분 소요)")
!git clone https://huggingface.co/lj1995/GPT-SoVITS pretrained_models_temp

# 파일 이동
!mv pretrained_models_temp/*.ckpt GPT_SoVITS/pretrained_models/ 2>/dev/null || true
!mv pretrained_models_temp/*.pth GPT_SoVITS/pretrained_models/ 2>/dev/null || true
!rm -rf pretrained_models_temp

print("\n✅ 모델 다운로드 완료!")
!ls -lh GPT_SoVITS/pretrained_models/

# ========================================
# STEP 7: 데이터 확인
# ========================================

print("\n" + "="*60)
print("STEP 7: 데이터 확인")
print("="*60)

# 이전에 준비한 데이터 확인
!ls -lh /content/GPT-SoVITS/sliced/

print("\n✅ GPT-SoVITS 설치 완료!")
print("\n다음 단계: ASR 실행")

STEP 0: 환경 확인

✓ Python 버전: 2.9.0+cu126
✓ CUDA 사용 가능: True
✓ GPU: Tesla T4
✓ CUDA 버전: 12.6

STEP 1: Python 3.10 설치
Python 3.12.12

STEP 2: FFmpeg 설치
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 66 not upgraded.
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers

STEP 3: GPT-SoVITS 다운로드
fatal: destination path 'GPT-SoVITS' already exists and is not an empty directory.
/content/GPT-SoVITS

STEP 4: PyTorch 설치
Looking in indexes: https://download.pytorch.org/whl/cu121
ERROR: Could not find a version that satisfies the requirement torch==2.1.0 (from versions: 2.2.0+cu121, 2.2.1+cu121, 2.2.2+cu121, 2.3.0+cu121, 2

In [9]:
# 기존 GPT-SoVITS 폴더 이름 변경 (데이터 보존)
!mv /content/GPT-SoVITS /content/GPT-SoVITS_DATA

# GPT-SoVITS 저장소 클론
!git clone https://github.com/RVC-Boss/GPT-SoVITS.git /content/GPT-SoVITS

# 클론 확인
!ls -la /content/GPT-SoVITS/

# 데이터 다시 복사
!cp -r /content/GPT-SoVITS_DATA/dataset /content/GPT-SoVITS/
!cp -r /content/GPT-SoVITS_DATA/sliced /content/GPT-SoVITS/
!cp /content/GPT-SoVITS_DATA/character_references.json /content/GPT-SoVITS/

print("\n✅ GPT-SoVITS 재설치 완료!")

Cloning into '/content/GPT-SoVITS'...
remote: Enumerating objects: 5780, done.
remote: Total 5780 (delta 0), reused 0 (delta 0), pack-reused 5780 (from 1)
Receiving objects: 100% (5780/5780), 14.01 MiB | 17.00 MiB/s, done.
Resolving deltas: 100% (3320/3320), done.
total 312
drwxr-xr-x  8 root root  4096 Dec  8 05:13 .
drwxr-xr-x  1 root root  4096 Dec  8 05:13 ..
-rw-r--r--  1 root root 46113 Dec  8 05:13 api.py
-rw-r--r--  1 root root 22169 Dec  8 05:13 api_v2.py
-rw-r--r--  1 root root  5060 Dec  8 05:13 Colab-Inference.ipynb
-rw-r--r--  1 root root  2764 Dec  8 05:13 Colab-WebUI.ipynb
-rw-r--r--  1 root root  7773 Dec  8 05:13 config.py
drwxr-xr-x  2 root root  4096 Dec  8 05:13 Docker
-rw-r--r--  1 root root  1775 Dec  8 05:13 docker_build.sh
-rw-r--r--  1 root root  1885 Dec  8 05:13 docker-compose.yaml
-rw-r--r--  1 root root  1584 Dec  8 05:13 Dockerfile
-rw-r--r--  1 root root  3689 Dec  8 05:13 .dockerignore
drwxr-xr-x  7 root root  4096 Dec  8 05:13 docs
-rw-r--r--  1 root ro

In [1]:
# 최신 PyTorch 설치 (2.5.1)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# 나머지는 그대로 진행
!pip install -r requirements.txt

# 추가 패키지
!pip install cn2an pypinyin funasr==1.0.27

# CUDA 확인
import torch
print(f"\n✓ PyTorch: {torch.__version__}")
print(f"✓ CUDA: {torch.cuda.is_available()}")

print("\n✅ 의존성 설치 완료!")

Looking in indexes: https://download.pytorch.org/whl/cu121
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

✓ PyTorch: 2.9.0+cu126
✓ CUDA: True

✅ 의존성 설치 완료!


In [3]:
# 필수 패키지 직접 설치
!pip install numpy scipy librosa soundfile
!pip install gradio fastapi uvicorn
!pip install tensorboard
!pip install cn2an pypinyin
!pip install funasr==1.0.27
!pip install onnxruntime-gpu

print("\n✅ 필수 패키지 설치 완료!")


✅ 필수 패키지 설치 완료!


In [4]:
# 사전 학습 모델 다운로드
!mkdir -p GPT_SoVITS/pretrained_models
!apt-get install git-lfs -y -qq
!git lfs install

print("\n⏳ 모델 다운로드 중... (3-5분 소요)")
!git clone https://huggingface.co/lj1995/GPT-SoVITS pretrained_temp

# 파일 이동
!mv pretrained_temp/*.ckpt GPT_SoVITS/pretrained_models/ 2>/dev/null || true
!mv pretrained_temp/*.pth GPT_SoVITS/pretrained_models/ 2>/dev/null || true
!rm -rf pretrained_temp

!ls -lh GPT_SoVITS/pretrained_models/

Git LFS initialized.

⏳ 모델 다운로드 중... (3-5분 소요)
Cloning into 'pretrained_temp'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 69 (delta 0), reused 0 (delta 0), pack-reused 68 (from 2)
Unpacking objects: 100% (69/69), 115.74 KiB | 1.23 MiB/s, done.
Filtering content: 100% (19/19), 4.93 GiB | 52.05 MiB/s, done.
total 1.2G
-rw-r--r-- 1 root root 148M Dec  8 05:20 's1bert25hz-2kh-longer-epoch=68e-step=50232.ckpt'
-rw-r--r-- 1 root root 149M Dec  8 05:20  s1v3.ckpt
-rw-r--r-- 1 root root  90M Dec  8 05:21  s2D488k.pth
-rw-r--r-- 1 root root 102M Dec  8 05:20  s2G488k.pth
-rw-r--r-- 1 root root 734M Dec  8 05:21  s2Gv3.pth


In [10]:
# faster-whisper 설치
!pip install faster-whisper

print("\n✅ faster-whisper 설치 완료!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 118.3 MB/s eta 0:00:00

✅ faster-whisper 설치 완료!


In [14]:
# PYTHONPATH 설정 후 ASR 실행
import os
os.environ['PYTHONPATH'] = '/content/GPT-SoVITS:' + os.environ.get('PYTHONPATH', '')

import subprocess

characters = [
    "boy", "scholar", "nolbu", "king", "commander", "bachelor",
    "teacher", "librarian", "queen", "girl", "grandma", "princess"
]

import os
os.environ['PYTHONPATH'] = '/content/GPT-SoVITS'

for char in characters:
    print(f'\n🗣️ {char} ASR 진행 중...')

    cmd = [
        'python', 'tools/asr/fasterwhisper_asr.py',
        '-i', f'sliced/{char}',
        '-o', f'asr_output/{char}',
        '-s', 'large-v3',
        '-l', 'ko',
        '-p', 'float16'
    ]

    subprocess.run(cmd, cwd='/content/GPT-SoVITS')

    print(f'✅ {char} ASR 완료')

print('\n✅ 모든 캐릭터 ASR 완료!')


🗣️ boy ASR 진행 중...
✅ boy ASR 완료

🗣️ scholar ASR 진행 중...
✅ scholar ASR 완료

🗣️ nolbu ASR 진행 중...
✅ nolbu ASR 완료

🗣️ king ASR 진행 중...
✅ king ASR 완료

🗣️ commander ASR 진행 중...
✅ commander ASR 완료

🗣️ bachelor ASR 진행 중...
✅ bachelor ASR 완료

🗣️ teacher ASR 진행 중...
✅ teacher ASR 완료

🗣️ librarian ASR 진행 중...
✅ librarian ASR 완료

🗣️ queen ASR 진행 중...
✅ queen ASR 완료

🗣️ girl ASR 진행 중...
✅ girl ASR 완료

🗣️ grandma ASR 진행 중...
✅ grandma ASR 완료

🗣️ princess ASR 진행 중...
✅ princess ASR 완료

✅ 모든 캐릭터 ASR 완료!


In [16]:
# ========================================
# 학습 데이터 준비
# ========================================

# 1. 모든 캐릭터의 .list 파일 통합
characters = [
    "boy", "scholar", "nolbu", "king", "commander", "bachelor",
    "teacher", "librarian", "queen", "girl", "grandma", "princess"
]

# 학습용 통합 리스트 생성
all_lines = []

for char in characters:
    list_file = f"/content/GPT-SoVITS/asr_output/{char}/{char}.list"

    try:
        with open(list_file, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            all_lines.extend(lines)
            print(f"✓ {char}: {len(lines)}개 문장")
    except:
        print(f"⚠️ {char}: 파일 없음")

# 통합 리스트 저장
output_dir = "/content/GPT-SoVITS/logs/prepared"
!mkdir -p {output_dir}

with open(f"{output_dir}/train.list", 'w', encoding='utf-8') as f:
    f.writelines(all_lines)

print(f"\n✅ 총 {len(all_lines)}개 문장 준비 완료")
print(f"📁 저장 위치: {output_dir}/train.list")

# 2. 통계 확인
!wc -l {output_dir}/train.list

✓ boy: 7개 문장
✓ scholar: 8개 문장
✓ nolbu: 5개 문장
✓ king: 4개 문장
✓ commander: 10개 문장
✓ bachelor: 7개 문장
✓ teacher: 7개 문장
✓ librarian: 14개 문장
✓ queen: 4개 문장
✓ girl: 13개 문장
✓ grandma: 5개 문장
✓ princess: 8개 문장

✅ 총 92개 문장 준비 완료
📁 저장 위치: /content/GPT-SoVITS/logs/prepared/train.list
80 /content/GPT-SoVITS/logs/prepared/train.list


# 여기서 부터 본격적 학습

In [27]:
# 깨끗하게 재설치
!pip uninstall peft -y -q
!pip install peft==0.13.2 -q

print("✅ peft 재설치 완료")

# 런타임 재시작 필요 (import 캐시 문제)
print("\n⚠️ 런타임 재시작이 필요합니다.")
print("상단 메뉴: 런타임 > 런타임 다시 시작")
print("\n재시작 후 다음 코드를 실행하세요:")
print("""
# 런타임 재시작 후 실행할 코드
import os
os.chdir('/content/GPT-SoVITS')

!nohup python api_v2.py > api_server.log 2>&1 &

import time
time.sleep(30)

# 확인
import requests
response = requests.get("http://127.0.0.1:9880")
print(f"✅ 서버 응답: {response.status_code}")
""")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 146.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 117.8 MB/s eta 0:00:00
✅ peft 재설치 완료

⚠️ 런타임 재시작이 필요합니다.
상단 메뉴: 런타임 > 런타임 다시 시작

재시작 후 다음 코드를 실행하세요:

# 런타임 재시작 후 실행할 코드
import os
os.chdir('/content/GPT-SoVITS')

!nohup python api_v2.py > api_server.log 2>&1 &

import time
time.sleep(30)

# 확인
import requests
response = requests.get("http://127.0.0.1:9880")
print(f"✅ 서버 응답: {response.status_code}")



In [5]:
# PYTHONPATH 설정
import sys
import os

os.chdir('/content/GPT-SoVITS')
sys.path.insert(0, '/content/GPT-SoVITS')
sys.path.insert(0, '/content/GPT-SoVITS/GPT_SoVITS')

# 모델 파일만 먼저 확인
print("📂 사전학습 모델 파일 확인:\n")
!ls -lh GPT_SoVITS/pretrained_models/*.ckpt 2>/dev/null || echo "ckpt 없음"
!ls -lh GPT_SoVITS/pretrained_models/*.pth 2>/dev/null || echo "pth 없음"

print("\n📂 전체 pretrained_models 폴더:")
!ls -la GPT_SoVITS/pretrained_models/

📂 사전학습 모델 파일 확인:

ckpt 없음
pth 없음

📂 전체 pretrained_models 폴더:
total 12
drwxr-xr-x  2 root root 4096 Dec  8 05:13 .
drwxr-xr-x 14 root root 4096 Dec  8 05:42 ..
-rw-r--r--  1 root root   13 Dec  8 05:13 .gitignore


In [6]:
# 사전학습 모델 다운로드
import os
os.chdir('/content/GPT-SoVITS')

print("📥 사전학습 모델 다운로드 중...")
print("⏳ 약 3-5분 소요 (3GB)")

# Git LFS 설치
!apt-get install git-lfs -y -qq
!git lfs install

# HuggingFace에서 모델 다운로드
!git clone https://huggingface.co/lj1995/GPT-SoVITS GPT_SoVITS/pretrained_models_temp

# .ckpt와 .pth 파일만 이동
!mkdir -p GPT_SoVITS/pretrained_models
!mv GPT_SoVITS/pretrained_models_temp/*.ckpt GPT_SoVITS/pretrained_models/ 2>/dev/null || true
!mv GPT_SoVITS/pretrained_models_temp/*.pth GPT_SoVITS/pretrained_models/ 2>/dev/null || true

# 정리
!rm -rf GPT_SoVITS/pretrained_models_temp

# 확인
print("\n✅ 다운로드 완료! 파일 확인:")
!ls -lh GPT_SoVITS/pretrained_models/

📥 사전학습 모델 다운로드 중...
⏳ 약 3-5분 소요 (3GB)
Updated git hooks.
Git LFS initialized.
Cloning into 'GPT_SoVITS/pretrained_models_temp'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 69 (delta 11), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (69/69), 116.16 KiB | 5.28 MiB/s, done.
Filtering content: 100% (19/19), 4.93 GiB | 53.85 MiB/s, done.

✅ 다운로드 완료! 파일 확인:
total 1.2G
-rw-r--r-- 1 root root 148M Dec  8 05:55 's1bert25hz-2kh-longer-epoch=68e-step=50232.ckpt'
-rw-r--r-- 1 root root 149M Dec  8 05:55  s1v3.ckpt
-rw-r--r-- 1 root root  90M Dec  8 05:55  s2D488k.pth
-rw-r--r-- 1 root root 102M Dec  8 05:55  s2G488k.pth
-rw-r--r-- 1 root root 734M Dec  8 05:56  s2Gv3.pth


In [15]:
# CNHuBERT 모델 다운로드
!mkdir -p GPT_SoVITS/pretrained_models/chinese-hubert-base
!git clone https://huggingface.co/TencentGameMate/chinese-hubert-base GPT_SoVITS/pretrained_models/chinese-hubert-base

print("✅ CNHuBERT 다운로드 완료")

# BERT도 로컬로 다운로드
!mkdir -p GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large
!git clone https://huggingface.co/hfl/chinese-roberta-wwm-ext-large GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large

print("✅ BERT 다운로드 완료")

# 이제 TTS 다시 초기화
import sys
import os
os.chdir('/content/GPT-SoVITS')
sys.path.insert(0, '/content/GPT-SoVITS')

from GPT_SoVITS.TTS_infer_pack.TTS import TTS, TTS_Config

tts_config = TTS_Config()
tts_config.t2s_weights_path = "GPT_SoVITS/pretrained_models/s1bert25hz-2kh-longer-epoch=68e-step=50232.ckpt"
tts_config.vits_weights_path = "GPT_SoVITS/pretrained_models/s2G488k.pth"
tts_config.bert_base_path = "GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large"
tts_config.cnhuhbert_base_path = "GPT_SoVITS/pretrained_models/chinese-hubert-base"
tts_config.device = "cuda"
tts_config.is_half = True
tts_config.version = "v1"

print("\n🎙️ TTS 초기화 중...")
tts_pipeline = TTS(tts_config)

print("✅ TTS 초기화 완료!")

Cloning into 'GPT_SoVITS/pretrained_models/chinese-hubert-base'...
remote: Enumerating objects: 22, done.
remote: Total 22 (delta 0), reused 0 (delta 0), pack-reused 22 (from 1)
Unpacking objects: 100% (22/22), 3.37 KiB | 689.00 KiB/s, done.
Filtering content: 100% (2/2), 1.41 GiB | 33.30 MiB/s, done.
✅ CNHuBERT 다운로드 완료
Cloning into 'GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large'...
remote: Enumerating objects: 50, done.
remote: Total 50 (delta 0), reused 0 (delta 0), pack-reused 50 (from 1)
Unpacking objects: 100% (50/50), 156.68 KiB | 4.61 MiB/s, done.
Filtering content: 100% (3/3), 3.64 GiB | 28.22 MiB/s, done.
✅ BERT 다운로드 완료
fall back to default t2s_weights_path: GPT_SoVITS/pretrained_models/gsv-v2final-pretrained/s1bert25hz-5kh-longer-epoch=12-step=369668.ckpt
fall back to default vits_weights_path: GPT_SoVITS/pretrained_models/gsv-v2final-pretrained/s2G2333k.pth

🎙️ TTS 초기화 중...
Loading Text2Semantic weights from GPT_SoVITS/pretrained_models/s1bert25hz-2kh-longer-epo

오디오북 생성 전 테스트

In [18]:
# 참조 오디오 확인 및 수정
print("📂 참조 오디오 파일 확인:")
ref_audio = "/content/GPT-SoVITS/sliced/teacher/teacher_0000.wav"

# 파일 존재 확인
import os
if os.path.exists(ref_audio):
    print(f"✅ 파일 존재: {ref_audio}")

    # 오디오 길이 확인
    import librosa
    y, sr = librosa.load(ref_audio, sr=None)
    duration = librosa.get_duration(y=y, sr=sr)
    print(f"  길이: {duration:.2f}초")
    print(f"  샘플레이트: {sr}Hz")
else:
    print(f"❌ 파일 없음: {ref_audio}")

# ASR로 생성된 실제 텍스트 확인
print("\n📝 ASR 생성 텍스트 확인:")
asr_file = "/content/GPT-SoVITS/asr_output/teacher/teacher.list"
with open(asr_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    if lines:
        first_line = lines[0].strip()
        parts = first_line.split('|')
        if len(parts) >= 4:
            actual_ref_text = parts[3]
            print(f"실제 텍스트: {actual_ref_text}")
        else:
            print(f"전체 라인: {first_line}")

# 실제 텍스트로 다시 시도
print("\n🎤 테스트 생성...")

try:
    gen_audio = tts_pipeline.run(
        ref_wav_path=ref_audio,
        prompt_text=actual_ref_text,  # ASR에서 추출한 실제 텍스트
        prompt_lang="ko",
        text="옛날 옛날에 한 공주님이 아름다운 성에 살고 계셨어요.",
        text_lang="ko"
    )

    import soundfile as sf
    sf.write("/content/test_success.wav", gen_audio, 32000)
    print("✅ 테스트 성공!")

    from IPython.display import Audio, display
    display(Audio("/content/test_success.wav"))

except Exception as e:
    print(f"❌ 에러: {e}")
    import traceback
    traceback.print_exc()

📂 참조 오디오 파일 확인:
✅ 파일 존재: /content/GPT-SoVITS/sliced/teacher/teacher_0000.wav
  길이: 9.04초
  샘플레이트: 44100Hz

📝 ASR 생성 텍스트 확인:
실제 텍스트:  그녀는 남녀를 막론하고 노예들에게 먹을 것과 마실 것을 베풀어 주었고, 그들의 능력에 따라서 이를 분배해 주었습니다.

🎤 테스트 생성...
❌ 에러: tuple index out of range


Traceback (most recent call last):
  File "/tmp/ipython-input-1873960118.py", line 46, in <cell line: 0>
    sf.write("/content/test_success.wav", gen_audio, 32000)
  File "/usr/local/lib/python3.12/dist-packages/soundfile.py", line 362, in write
    channels = data.shape[1]
               ~~~~~~~~~~^^^
IndexError: tuple index out of range


In [21]:
# TTS.run() 메서드 시그니처 확인
import inspect

print("🔍 TTS.run() 메서드 시그니처:\n")
sig = inspect.signature(tts_pipeline.run)
print(sig)

print("\n📋 파라미터 목록:")
for param_name, param in sig.parameters.items():
    print(f"  - {param_name}: {param.annotation if param.annotation != inspect.Parameter.empty else 'Any'}")

# 소스 코드 확인
print("\n📄 메서드 독스트링:")
print(tts_pipeline.run.__doc__)

🔍 TTS.run() 메서드 시그니처:

(inputs: dict)

📋 파라미터 목록:
  - inputs: <class 'dict'>

📄 메서드 독스트링:

        Text to speech inference.

        Args:
            inputs (dict):
                {
                    "text": "",                   # str.(required) text to be synthesized
                    "text_lang: "",               # str.(required) language of the text to be synthesized
                    "ref_audio_path": "",         # str.(required) reference audio path
                    "aux_ref_audio_paths": [],    # list.(optional) auxiliary reference audio paths for multi-speaker tone fusion
                    "prompt_text": "",            # str.(optional) prompt text for the reference audio
                    "prompt_lang": "",            # str.(required) language of the prompt text for the reference audio
                    "top_k": 5,                   # int. top k sampling
                    "top_p": 1,                   # float. top p sampling
                    "temperature"

In [24]:
# 지원 언어 확인
print("🌍 지원 언어 목록:")
print(tts_pipeline.configs.languages)

# 한국어 코드 찾기
for lang in tts_pipeline.configs.languages:
    print(f"  - {lang}")

🌍 지원 언어 목록:
['auto', 'en', 'zh', 'ja', 'all_zh', 'all_ja']
  - auto
  - en
  - zh
  - ja
  - all_zh
  - all_ja


In [26]:
# torchcodec 설치
!pip install torchcodec -q

print("✅ torchcodec 설치 완료")

# TTS 다시 실행
print("\n🎤 TTS 재시도...\n")

inputs = {
    "text": "안녕하세요. 테스트입니다.",
    "text_lang": "zh",
    "ref_audio_path": "/content/GPT-SoVITS/sliced/teacher/teacher_0000.wav",
    "prompt_text": "그녀는 남녀를 막론하고 노예들에게 먹을 것과 마실 것을 베풀어 주었고, 그들의 능력에 따라서 이를 분배해 주었습니다.",
    "prompt_lang": "zh",
    "streaming_mode": False,
    "return_fragment": False
}

try:
    result_gen = tts_pipeline.run(inputs)

    for result in result_gen:
        if isinstance(result, tuple) and len(result) == 2:
            sr, audio_data = result
            print(f"✅ 생성 성공!")
            print(f"  샘플링레이트: {sr}Hz")
            print(f"  오디오 길이: {len(audio_data)/sr:.2f}초")

            # 저장
            import soundfile as sf
            sf.write("/content/test_korean.wav", audio_data, sr)

            from IPython.display import Audio, display
            display(Audio("/content/test_korean.wav"))
            break

except Exception as e:
    print(f"❌ 에러: {e}")
    import traceback
    traceback.print_exc()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.5 MB/s eta 0:00:00
✅ torchcodec 설치 완료

🎤 TTS 재시도...

Set seed to 252021694
Parallel Inference Mode Enabled
Bucket Processing Mode Enabled
Actual Input Reference Text: 그녀는 남녀를 막론하고 노예들에게 먹을 것과 마실 것을 베풀어 주었고, 그들의 능력에 따라서 이를 분배해 주었습니다.
############ Segment Text ############
Actual Input Target Text:
안녕하세요. 테스트입니다.
Actual Input Target Text (after sentence segmentation):
['안녕하세요. 테스트입니다.']
############ Extract Text BERT Features ############


100%|██████████| 1/1 [00:00<00:00, 27.26it/s]


############ 推理 ############
Processed text from the frontend (per sentence): ['.']
############ Predict Semantic Token ############


  0%|          | 1/1500 [00:00<07:26,  3.36it/s]


T2S Decoding EOS [240 -> 242]
############ Synthesize Audio ############
Parallel Synthesis in Progress...
2.451	0.046	0.383	1.299
✅ 생성 성공!
  샘플링레이트: 32000Hz
  오디오 길이: 0.34초


오디오북 생성

In [30]:
# 🎙️ 전체 동화 오디오북 생성
import json
import soundfile as sf
from pathlib import Path

# JSON 파일 로드
json_path = "/content/유민이의_특별한_한복_찾기_20251207_2253.json"

with open(json_path, 'r', encoding='utf-8') as f:
    story_data = json.load(f)

print(f"📖 동화 제목: {story_data['title']}")
print(f"📝 총 {len(story_data['script'])}개 세그먼트\n")

# 출력 디렉토리
output_dir = "/content/audiobook_output"
Path(output_dir).mkdir(exist_ok=True)

# Narrator 참조
ref_audio = "/content/GPT-SoVITS/sliced/teacher/teacher_0000.wav"
ref_text = "그녀는 남녀를 막론하고 노예들에게 먹을 것과 마실 것을 베풀어 주었고, 그들의 능력에 따라서 이를 분배해 주었습니다."

print("🎬 오디오북 생성 시작...\n")

for segment in story_data['script']:
    seq = segment['seq']
    text = segment['text']
    audio_filename = segment['audio_file_name']

    print(f"[{seq}/8] 생성 중: {text[:40]}...")

    try:
        inputs = {
            "text": text,
            "text_lang": "zh",  # 한국어를 zh로 처리
            "ref_audio_path": ref_audio,
            "prompt_text": ref_text,
            "prompt_lang": "zh",
            "streaming_mode": False,
            "return_fragment": False
        }

        result_gen = tts_pipeline.run(inputs)

        for result in result_gen:
            if isinstance(result, tuple) and len(result) == 2:
                sr, audio_data = result

                # 저장
                output_path = f"{output_dir}/{audio_filename}"
                sf.write(output_path, audio_data, sr)

                duration = len(audio_data) / sr
                print(f"  ✅ 완료 ({duration:.1f}초): {audio_filename}\n")
                break

    except Exception as e:
        print(f"  ❌ 오류: {e}\n")

print("="*60)
print("✅ 오디오북 생성 완료!")
print(f"📁 저장 위치: {output_dir}\n")

# 생성된 파일 확인
!ls -lh {output_dir}

# 첫 번째 파일 재생
from IPython.display import Audio, display
print("\n🎵 첫 번째 세그먼트 재생:")
display(Audio(f"{output_dir}/001_narrator.wav"))

📖 동화 제목: 유민이의 특별한 한복 찾기
📝 총 8개 세그먼트

🎬 오디오북 생성 시작...

[1/8] 생성 중: 높은 빌딩이 숲을 이루고 우리가 살고 있는 바로 지금, 서울 동대문 근처...
Set seed to 469456609
Parallel Inference Mode Enabled
Bucket Processing Mode Enabled
Actual Input Reference Text: 그녀는 남녀를 막론하고 노예들에게 먹을 것과 마실 것을 베풀어 주었고, 그들의 능력에 따라서 이를 분배해 주었습니다.
############ Segment Text ############
Actual Input Target Text:
높은 빌딩이 숲을 이루고 우리가 살고 있는 바로 지금, 서울 동대문 근처에 사는 유민이는 하루하루가 모험 같았어요. 유민이는 초등학교에 다니는 소녀로, 늘 공주 옷을 입고 싶어 했죠. 어느 날, 유민이는 엄마와 함께 동대문 시장을 방문했어요. 유민이의 엄마는 동대문 시장에서 한복 가게를 운영하는 따뜻하고 자상한 분이었답니다.
Actual Input Target Text (after sentence segmentation):
['높은 빌딩이 숲을 이루고 우리가 살고 있는 바로 지금, 서울 동대문 근처에 사는 유민이는 하루하루가 모험 같았어요. 유민이는 초등학교에 다니는 소녀로, 늘 공주 옷을 입고 싶어 했죠. 어느 날, 유민이는 엄마와 함께 동대문 시장을 방문했어요. 유민이의 엄마는 동대문 시장에서 한복 가게를 운영하는 따뜻하고 자상한 분이었답니다.']
############ Extract Text BERT Features ############


100%|██████████| 1/1 [00:00<00:00, 26.35it/s]


############ 推理 ############
Processed text from the frontend (per sentence): ['.']
############ Predict Semantic Token ############


  0%|          | 1/1500 [00:00<00:41, 36.06it/s]

T2S Decoding EOS [240 -> 242]
############ Synthesize Audio ############
Parallel Synthesis in Progress...
0.000	0.042	0.034	0.049


...

100%|██████████| 1/1 [00:00<00:00, 18.11it/s]


############ 推理 ############
Processed text from the frontend (per sentence): ['.']
############ Predict Semantic Token ############


  0%|          | 2/1500 [00:00<00:42, 35.59it/s]

T2S Decoding EOS [240 -> 243]
############ Synthesize Audio ############
Parallel Synthesis in Progress...


0.000	0.059	0.061	0.767
  ✅ 완료 (0.4초): 008_narrator.wav

✅ 오디오북 생성 완료!
📁 저장 위치: /content/audiobook_output

total 196K
-rw-r--r-- 1 root root 22K Dec  8 06:13 001_narrator.wav
-rw-r--r-- 1 root root 22K Dec  8 06:13 002_narrator.wav
-rw-r--r-- 1 root root 22K Dec  8 06:13 003_narrator.wav
-rw-r--r-- 1 root root 27K Dec  8 06:13 004_narrator.wav
-rw-r--r-- 1 root root 22K Dec  8 06:13 005_narrator.wav
-rw-r--r-- 1 root root 22K Dec  8 06:13 006_narrator.wav
-rw-r--r-- 1 root root 22K Dec  8 06:13 007_narrator.wav
-rw-r--r-- 1 root root 24K Dec  8 06:13 008_narrator.wav

🎵 첫 번째 세그먼트 재생:
